In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from QuadUtils import K_xi_from_Intrinsic


In [ ]:
image_gray = cv2.imread('sample_data/image_170_0.png', cv2.IMREAD_GRAYSCALE)
intrinsic_init = np.array([1.24, 813, 812, 640, 360])
D_init = np.array([-0.38871409,  0.14562629, -0.00313268, -0.0010537])

K, xi = K_xi_from_Intrinsic(intrinsic_init)
new_size = (1280, 720)
Knew = np.array([[400, 0.0, 0.0],
                [0.0, 300, 0.0],
                [0.0, 0.0, 1.0]], dtype=np.float64)
map1, map2 = cv2.omnidir.initUndistortRectifyMap(K, D_init, xi, np.eye(3),
                                                            Knew, new_size, cv2.CV_16SC2, cv2.omnidir.RECTIFY_LONGLATI)
image_undist = cv2.remap(image_gray, map1, map2, interpolation=cv2.INTER_LINEAR)

image_combined = np.hstack((image_gray, image_undist))
plt.imshow(image_combined, cmap='gray')
plt.show()